In [ ]:
import json, os, sys
%load_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"]="1"
# from utils import session_config
# session_config.setup_gpus(True, 0.95)

from matplotlib import pyplot as plt
import tensorflow as tf
import cv2
import numpy as np
from functools import partial
from utils.iou import compute_iou

# load image information from coco object

In [ ]:
sys.path.append("/home/files/cocoapi/PythonAPI")
from pycocotools.coco import COCO
coco = COCO("annotations/integrated_annotation.json")

train_ids = np.load("annotations/train_ids.npy")
print("train-data length : {}".format(len(train_ids)))
val_ids = np.load("annotations/val_ids.npy")
print("train-data length : {}".format(len(val_ids)))

In [ ]:
boxes = np.zeros([19,4])
for i in range(19):
    boxes[i] = [0,0+0.05*i,1,0.1+0.05*i]
def generate(coco, ids, boxes):
    for image_id in ids:   
        image = coco.loadImgs(int(image_id))
        image_array = cv2.imread("detection" + image[0]["path"])[:,:,::-1]
        anno_ids = coco.getAnnIds(imgIds=int(image_id), catIds=[0], iscrowd=None)
        anns = coco.loadAnns(anno_ids)

        image_array = image_array[np.newaxis,...]
        patches = tf.image.extract_patches(images=image_array,
                                   sizes=[1, 1000, 1024, 1],
                                   strides=[1, 500, 1, 1],
                                   rates=[1, 1, 1, 1],
                                   padding='VALID')
        patches = tf.squeeze(patches)
        patches = tf.reshape(patches, (19, 1000,1024,3))
        patches = tf.image.resize(patches, (224,224), method="area")
        bbox = []
        for ann in anns:
            x,y,w,h = ann["bbox"]
            x,y,w,h = x/2024,y/10000,w/2024,h/10000
            bbox.append([x,y,x+w,y+h])   
        bbox = np.array(bbox)        
        ious = compute_iou(bbox, boxes)
        ious = np.sum(ious, axis=0)
        ious = tf.squeeze(ious)
        labels = tf.where(ious>0, 1, 0)
        yield patches, labels

In [ ]:
def data_process(images, labels):
    images = tf.reshape(images, [-1,224,224,3])
    images = tf.cast(images, tf.float32)
    images = tf.keras.applications.resnet_v2.preprocess_input(images)
    labels = tf.reshape(labels, [-1])
    return images, labels

batch_size = 6; AUTOTUNE = tf.data.experimental.AUTOTUNE;

train_gen = partial(generate, coco, train_ids, boxes)
val_gen = partial(generate, coco, val_ids, boxes)
# generate data pipeline with from_generator in TensorFlow dataset APIs
train_ds = tf.data.Dataset.from_generator(train_gen, (tf.uint8, tf.int32))
val_ds = tf.data.Dataset.from_generator(val_gen, (tf.uint8, tf.int32), output_shapes=((19,224,224,3), (19,)))

# dataset pipeline
train_ds = train_ds.cache().shuffle(buffer_size=1000).batch(batch_size).map(data_process).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().batch(batch_size).map(data_process).prefetch(buffer_size=AUTOTUNE)

In [ ]:
for images, labels in train_ds.take(5):
    print(images.shape)
    print(labels.shape)

In [ ]:
for images, labels in val_ds.take(1):
    print(images.shape)
    print(labels.shape)

# 모델 빌드 및 학습

In [ ]:
resnet = tf.keras.applications.ResNet50V2(include_top=False, 
                                         weights='imagenet', 
                                         input_shape=(224,224,3))

In [ ]:
model = tf.keras.Sequential([resnet,
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(1, activation=None),
                             tf.keras.layers.Softmax()])                             

In [ ]:
@tf.function
def train_step(images, labels, model):
    with tf.GradientTape() as tape:
        preds = model(images, training=True)
        loss = loss_object(labels,preds)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    loss(loss)
    acc(labels, preds)
@tf.function
def test_step(images, labels, model):
    preds = model(images, training=False)
    loss = loss_object(labels,preds)
    test_loss(loss)
    test_acc(labels, preds)

In [ ]:
loss = tf.keras.metrics.Mean(name='train_loss')
acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
template = "Epoch {} : train loss {:5f}, accuracy {:.3f}, val loss {:5f}, val accuracy {:.3f}"

optimizer = tf.keras.optimizers.Adam(learning_rate=1E-3)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
EPOCH = 1
for epoch in range(1, EPOCH+1):
    loss.reset_states()
    acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()
    for images, labels in train_ds:
        print("one iteration")
        train_step(images, labels, model)
    for images, labels in val_ds:
        test_step(images, labels, model) 
    print(template.format(epoch,
                          loss.result().numpy(),
                          acc.result().numpy(),
                         test_loss.result().numpy(),
                         test_acc.result().numpy()))